# Complementaria Semana 9 (Jueves 12:30)

In [1]:
import numpy as np
import pandas as pd
from jmarkov.dtmc import dtmc

a) Estime la matriz de transición de probabilidades a partir del archivo $\texttt{aguila.xlsx}$

In [2]:
# Leer el archivo
df = pd.read_excel('datos (2).xlsx')

In [4]:
# Hint: Crear una nueva columna con la pareja estado inicial, estado futuro
df['transition'] = list(zip(df['previous_habitat'], df['current_habitat']))
unique_transitions = df['transition'].unique()
df

,season,previous_habitat,current_habitat,transition
0,1,Laguna,Laguna,"(Laguna, Laguna)"
1,2,Laguna,Pradera,"(Laguna, Pradera)"
2,3,Pradera,Laguna,"(Pradera, Laguna)"
3,4,Laguna,Laguna,"(Laguna, Laguna)"
4,5,Laguna,Pradera,"(Laguna, Pradera)"
...,...,...,...,...
495,496,Pradera,Pradera,"(Pradera, Pradera)"
496,497,Pradera,Pradera,"(Pradera, Pradera)"
497,498,Pradera,Pradera,"(Pradera, Pradera)"
498,499,Pradera,Pradera,"(Pradera, Pradera)"


In [6]:
# Estimar probabilidadea
state_counts = df.groupby("previous_habitat")["season"].count().to_dict()
state_counts

{'Bosque': 129, 'Laguna': 161, 'Pradera': 210}

In [7]:
transition_counts = df.groupby("transition")["season"].count().to_dict()
transition_counts

{('Bosque', 'Bosque'): 80,
 ('Bosque', 'Laguna'): 18,
 ('Bosque', 'Pradera'): 31,
 ('Laguna', 'Bosque'): 10,
 ('Laguna', 'Laguna'): 88,
 ('Laguna', 'Pradera'): 63,
 ('Pradera', 'Bosque'): 39,
 ('Pradera', 'Laguna'): 54,
 ('Pradera', 'Pradera'): 117}

In [8]:
empirical_probs = {}

for transition, count in transition_counts.items():
    empirical_probs[transition] = count / state_counts[transition[0]]
    
empirical_probs

{('Bosque', 'Bosque'): 0.6201550387596899,
 ('Bosque', 'Laguna'): 0.13953488372093023,
 ('Bosque', 'Pradera'): 0.24031007751937986,
 ('Laguna', 'Bosque'): 0.062111801242236024,
 ('Laguna', 'Laguna'): 0.546583850931677,
 ('Laguna', 'Pradera'): 0.391304347826087,
 ('Pradera', 'Bosque'): 0.18571428571428572,
 ('Pradera', 'Laguna'): 0.2571428571428571,
 ('Pradera', 'Pradera'): 0.5571428571428572}

b) Matriz de transición y cadena con la libreria jmarkov:

In [9]:
estados = np.array(['Bosque','Pradera','Laguna'])

matrizP = np.zeros((len(estados),len(estados)))

for fila, i in enumerate(estados):
    for columna, j in enumerate(estados):
        matrizP[fila,columna] = empirical_probs[i,j]

cadena = dtmc(matrizP)

c) Dentro de tres temporadas está programado un mantenimiento general en el Bosque. No obstante, si el ave se encuentra en el Bosque en esa temporada, el mantenimiento deberá cancelarse.
 Proporcione una expresión para calcular la probabilidad de que, en tres temporadas, se deba cancelar el mantenimiento, dado que en la temporada actual el ave se encuentra en la Laguna.

In [28]:
# transient probabilities

alpha = (estados=="Laguna").astype(float)
alpha

probabilidades_transitorias = cadena.transient_probabilities(n=3, alpha=alpha)
respuesta_c = probabilidades_transitorias[np.where(estados=="Bosque")[0][0]]


#respuesta_c = float(respuesta_c)
print(f"La probabilidad de estar en 'Bosque' en la temporada 3, partiendo de 'Laguna' es: {round(respuesta_c, 4)}")

La probabilidad de estar en 'Bosque' en la temporada 3, partiendo de 'Laguna' es: 0.1983


d) Para el Parque es necesario conocer cuánto tiempo en promedio durará cada estado durante los próximos 3 años. Cada año tiene 4 temporadas. Actualmente se está en la Laguna.


In [45]:
# occupation time

tiempos_ocupacion = cadena.occupation_time(3*4)

tiempo_ocupacion_laguna = tiempos_ocupacion[np.where(estados=="Laguna")[0][0]]

tiempo_ocupacion_laguna

suma = 0

for i, estado in enumerate(estados):
    print(f"El tiempo de ocupacion empezando de Laguna hasta {estado} es: {tiempo_ocupacion_laguna[i]}")
    suma += tiempo_ocupacion_laguna[i]

print("")
for i, estado in enumerate(estados):
    proporción = tiempo_ocupacion_laguna[i] / suma
    print(f"La proporción de tiempo en {estado} es: {proporción:.4f}")
    



El tiempo de ocupacion empezando de Laguna hasta Bosque es: 2.670656438348191
El tiempo de ocupacion empezando de Laguna hasta Pradera es: 5.108593068748927
El tiempo de ocupacion empezando de Laguna hasta Laguna es: 5.220750492902883

La proporción de tiempo en Bosque es: 0.2054
La proporción de tiempo en Pradera es: 0.3930
La proporción de tiempo en Laguna es: 0.4016


e) ¿Cuántas temporadas va a tardar el águila en estar por primera vez en la Pradera dado que actualmente está en la Laguna?

In [54]:
# Tiempo de primera pasada

for index, estadoo in enumerate(estados):
    print(f"Indice: {index}, Estado: {estadoo}")

tiempo_primera_pasada = cadena.first_passage_time(1)

tiempo_pp_laguna=tiempo_primera_pasada[1][0]

print(f"\nEl tiempo de primera pasada de Laguna a Pradera es: {round(tiempo_pp_laguna, 4)}")


Indice: 0, Estado: Bosque
Indice: 1, Estado: Pradera
Indice: 2, Estado: Laguna

El tiempo de primera pasada de Laguna a Pradera es: 2.7021
